In [ ]:
# importing necessary libraries
# pandas for etl, openpyxl for excel export, nsepython for data fetch from NSE
import pandas as pd
from datetime import date
from nsepython import *
import numpy as np
import openpyxl

In [ ]:
# assigning the dates for which application will run
end_date = (date.today() + datetime.timedelta(days=-1))
start_date = (date.today() + datetime.timedelta(days=-200))
delta = end_date - start_date   # returns timedelta

In [ ]:
# get_bhavcopy is the function to fetch delivery and total quantity
# https://unofficed.com/nse-python/documentation/nsepy/ ------official documentation

# Create an empty DataFrame
df_final = pd.DataFrame()

for i in range(delta.days + 1):
    day = start_date + datetime.timedelta(days=i)
    try:
        bhav_df = get_bhavcopy(day.strftime('%d-%m-%Y'))
        bhav_df = bhav_df.rename(columns=lambda x: x.strip())
        bhav_df['SERIES'] = bhav_df['SERIES'].str.strip()
        df_final = pd.concat([df_final, bhav_df], ignore_index=True)
        df_final = df_final[df_final['SERIES'] == 'EQ'] #filtering for 'EQ' as other serieses also coming
    except:
        print("This date is a holiday or weekend: ", day.strftime('%d-%m-%Y'))
        continue

In [97]:
# master data for getting full name per symbol
symbol_master = pd.read_excel('Stock_Symbol_Master_Data.xlsx')
# symbol_master

# creating an empty dataframe to store final results
final_df = pd.DataFrame()

In [98]:
for index, row in symbol_master.iterrows():
    # print(row['SYMBOL'])

    combined_df = df_final[(df_final['SYMBOL'] == row['SYMBOL'])]
    combined_df['DESCRIPTION'] = row['DESCRIPTION']
    combined_df = combined_df.drop_duplicates()
    # Convert 'Date' column from string to date
    combined_df['DATE1'] = pd.to_datetime(combined_df['DATE1'].str.strip(), format='%d-%b-%Y')
    combined_df = combined_df.sort_values(by=['SYMBOL','DATE1'])

    combined_df = combined_df[['DATE1','SYMBOL','DESCRIPTION','TTL_TRD_QNTY','NO_OF_TRADES','DELIV_QTY','DELIV_PER','CLOSE_PRICE']]
    combined_df[["DELIV_QTY", "DELIV_PER"]] = combined_df[["DELIV_QTY", "DELIV_PER"]].apply(pd.to_numeric)
    combined_df['change'] = combined_df['CLOSE_PRICE'].diff()

    mask = combined_df.SYMBOL != combined_df.SYMBOL.shift(1)
    combined_df['change'][mask] = np.nan

    combined_df['gain'] = combined_df.change.mask(combined_df.change < 0, 0.0)
    combined_df['loss'] = -combined_df.change.mask(combined_df.change > 0, -0.0)
    def rma(x, n):
        """Running moving average"""
        a = np.full_like(x, np.nan)
        a[n] = x[1:n+1].mean()
        for i in range(n+1, len(x)):
            a[i] = (a[i-1] * (n - 1) + x[i]) / n
        return a

    combined_df['avg_gain'] = rma(combined_df.gain.to_numpy(), 14)
    combined_df['avg_loss'] = rma(combined_df.loss.to_numpy(), 14)

    combined_df['rs'] = combined_df.avg_gain / combined_df.avg_loss
    combined_df['rsi'] = 100 - (100 / (1 + combined_df.rs))
    combined_df['AVG_TOTAL_TRADE_QTY'] = combined_df.rolling(window=90)['TTL_TRD_QNTY'].mean()

    final_df = pd.concat([final_df,combined_df], ignore_index=True)
    # print(len(final_df))

C:\Users\Krish\AppData\Local\Temp\ipykernel_28916\390505409.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['DESCRIPTION'] = row['DESCRIPTION']
C:\Users\Krish\AppData\Local\Temp\ipykernel_28916\390505409.py:16: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc

In [99]:
# renaming columns, dropping unnecessary columns and adding BUY FLAG
final_df = final_df.rename(columns={'DATE1': 'DATE', 'TTL_TRD_QNTY': 'TOTAL_TRADE_QTY', 'DELIV_QTY': 'DELIVERY_QTY',
                                    'DELIV_PER': 'DELIVERY_PERC', 'rs': 'RS', 'rsi': 'RSI'})
final_df = final_df.drop(['change', 'gain', 'loss', 'avg_gain', 'avg_loss'], axis=1)
final_df['BUY_FLAG'] = np.where((final_df['TOTAL_TRADE_QTY'] >= final_df['AVG_TOTAL_TRADE_QTY']) & (final_df['RSI'] >= 55), 'Yes', 'No')

In [100]:
rsi_56_df = final_df[(final_df['DATE'] == final_df['DATE'].max()) & (final_df['RSI'] >= 55)]
rsi_56_df

,DATE,DESCRIPTION,SYMBOL,TOTAL_TRADE_QTY,NO_OF_TRADES,DELIVERY_QTY,DELIVERY_PERC,CLOSE_PRICE,RS,RSI,AVG_TOTAL_TRADE_QTY,BUY_FLAG
134,2024-07-05,20 Microns Limited,20MICRONS,201028,7550,88660,44.10,224.72,2.033562,67.035449,3.277336e+05,No
269,2024-07-05,360 ONE WAM LIMITED,360ONE,498527,38057,210079,42.14,964.70,2.193409,68.685500,7.485813e+05,No
404,2024-07-05,3i Infotech Limited,3IINFOLTD,3389820,19174,1236757,36.48,40.21,1.707971,63.071976,1.392144e+06,Yes
539,2024-07-05,3M India Limited,3MINDIA,8303,3894,3627,43.68,38768.10,1.950547,66.107976,9.271167e+03,No
1028,2024-07-05,Aarti Drugs Limited,AARTIDRUGS,392716,24928,174149,44.34,554.45,2.192142,68.673072,1.863479e+05,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
207103,2024-07-05,Zota Health Care LImited,ZOTA,27178,2211,15621,57.48,594.00,1.703585,63.012080,5.234758e+04,No
207238,2024-07-05,Zuari Agro Chemicals Limited,ZUARI,220298,8248,111331,50.54,221.20,1.587516,61.352889,3.502460e+05,No
207373,2024-07-05,ZUARI INDUSTRIES LIMITED,ZUARIIND,162292,9678,69543,42.85,388.20,1.594664,61.459358,2.401172e+05,No
207508,2024-07-05,Zydus Lifesciences Limited,ZYDUSLIFE,2734011,105113,1024350,37.47,1162.45,2.365305,70.285009,1.528218e+06,Yes


In [104]:
def highlight_values(val):
    color = 'yellow' if val == 'Yes' else 'white'
    return f'background-color: {color}'

In [108]:
# create a excel writer object
with pd.ExcelWriter("StockMarketData.xlsx") as writer:
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    final_df.style.applymap(highlight_values, subset=['BUY_FLAG']).to_excel(writer, sheet_name="Complete_Data", index=False)
    rsi_56_df.to_excel(writer, sheet_name="RSI_55", index=False)

print('DataFrame is written to Excel File successfully.')

C:\Users\Krish\AppData\Local\Temp\ipykernel_28916\2886220481.py:5: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  final_df.style.applymap(highlight_values, subset=['BUY_FLAG']).to_excel(writer, sheet_name="Complete_Data", index=False)
Exception ignored in: <function ZipFile.__del__ at 0x0000017BB3A9A3E0>
Traceback (most recent call last):
  File "c:\Users\Krish\AppData\Local\Programs\Python\Python312\Lib\zipfile\__init__.py", line 1929, in __del__
    self.close()
  File "c:\Users\Krish\AppData\Local\Programs\Python\Python312\Lib\zipfile\__init__.py", line 1946, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
Exception ignored in: <function ZipFile.__del__ at 0x0000017BB3A9A3E0>
Traceback (most recent call last):
  File "c:\Users\Krish\AppData\Local\Programs\Python\Python312\Lib\zipfile\__init__.py", line 1929, in __del__
    self.close()
  File "c:\Users\Krish\AppData\Local\Programs\Python\Python312\Lib\zipfile\__init__.py", lin